In [1]:
## this notebook will deteck anomalies from the pcap outputs.

import pandas as pd
import numpy as np

from torch import nn

import scipy
import matplotlib.pyplot as plt


from data_extractors import advanced_process, basic_process

import os
import sys

##install pytorch

##install scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.feature_selection import SelectKBest
## train test split import with cross fold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


In [4]:
current_wd = os.getcwd()

three_days_advanced_raw = advanced_process(f"{current_wd}/pcap-files","three_days_adv.csv") ## this will likely cause errors now because the package is still getting collected in the background.

three_days_basic_raw = basic_process(f"{current_wd}/pcap-files","three_days_basic.csv") 


adv_df = pd.read_csv(three_days_advanced_raw)
basic_df = pd.read_csv(three_days_basic_raw)



# i will write a loop to get the statşstics of the data


def get_stats(dataframe = pd.DataFrame):

    """
    This function returns the statistics of the given dataframe.

    Parameters:
    dataframe: pd.DataFrame
        The dataframe to be analyzed.
    """
    try :
        display("--------------------")
        display("dataframe\'s shape is :")
        display(dataframe.shape)
        display("--------------------")
        display("dataframe\'s columns are :")
        display(dataframe.columns)
        display("--------------------")
        display("dataframe\'s info is :")
        display(dataframe.info())
        display("--------------------")
        display("dataframe\'s head is :")
        display(dataframe.head())
        display("--------------------")
        display("dataframe\'s tail is :")
        display(dataframe.tail())
        display("--------------------")
        display("dataframe\'s describe is :")
        display(dataframe.describe())
        display("--------------------")
        display("dataframe\'s null values are :")
        display(dataframe.isnull().sum())
        display("--------------------")
        display("dataframe\'s unique values are :")
        display(dataframe.nunique())
        display("--------------------")
        display("dataframe\'s data types are :")
        display(dataframe.dtypes)
        display("--------------------")
        display("dataframe\'s memory usage is :")
        display(dataframe.memory_usage())
        display("--------------------")

        ##scipy stats of the dataframe
        display("dataframe\'s column stats are :")
        ##skewness of the every column
        for col in dataframe.columns:
            #skewness
            display(f"{col} skewness is : {scipy.stats.skew(dataframe[col])}")
            #kurtosis
            display(f"{col} kurtosis is : {scipy.stats.kurtosis(dataframe[col])}")

            dataframe = pd.DataFrame(
                columns=["column name:","skewness", "kurtosis"],
                data = [
                    [col.name,scipy.stats.skew(dataframe[col]), scipy.stats.kurtosis(dataframe[col])]
                ]
            )



        display("--------------------")
        ##correlation of the columns
        display("dataframe\'s correlation is :")
        display(dataframe.corr())
        display("--------------------")
        ##covariance of the columns
        display("dataframe\'s covariance is :")
        display(dataframe.cov())
        display("--------------------")
        ## chi scores
        display("dataframe\'s chi scores are :")
        display(dataframe.chisquare())
        display("--------------------")

    except Exception as e:
        print("An error occured: ", e)
        return []


Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day3.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day2.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day1.pcap...
Successfully extracted features from 182 time windows
Data saved to three_days_adv.csv
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day3.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day2.pcap...
Processing /Users/eren/Documents/GitHub/Network-Anomaly-Detection/pcap-files/baseline_day1.pcap...
Successfully extracted features from 182 time windows
Data saved to three_days_basic.csv


In [3]:
get_stats(adv_df)

display(100*"-")

get_stats(basic_df)

'--------------------'

"dataframe's shape is :"

(174, 37)

'--------------------'

"dataframe's columns are :"

...

'unique_dst_ports kurtosis is : 6.775711801960757'

'avg_packet_size skewness is : 0.5559802027089125'

'avg_packet_size kurtosis is : -0.2044743299171592'

An error occured:  numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


[]

In [22]:


def get_stats(dataframe = pd.DataFrame):

    """
    This function returns the statistics of the given dataframe.

    Parameters:
    dataframe: pd.DataFrame
        The dataframe to be analyzed.
    """
    try :
        display("--------------------")
        display("dataframe\'s shape is :")
        display(dataframe.shape)
        display("--------------------")
        display("dataframe\'s columns are :")
        display(dataframe.columns)
        display("--------------------")
        display("dataframe\'s info is :")
        display(dataframe.info())
        display("--------------------")
        display("dataframe\'s head is :")
        display(dataframe.head())
        display("--------------------")
        display("dataframe\'s tail is :")
        display(dataframe.tail())
        display("--------------------")
        display("dataframe\'s describe is :")
        display(dataframe.describe())
        display("--------------------")
        display("dataframe\'s null values are :")
        display(dataframe.isnull().sum())
        display("--------------------")
        display("dataframe\'s unique values are :")
        display(dataframe.nunique())
        display("--------------------")
        display("dataframe\'s data types are :")
        display(dataframe.dtypes)
        display("--------------------")
        display("dataframe\'s memory usage is :")
        display(dataframe.memory_usage())
        display("--------------------")

        ##scipy stats of the dataframe
        display("dataframe\'s column stats are :")
        ##skewness of the every column
        for col in dataframe.columns:
            framey = pd.DataFrame(
                columns=["column name:","skewness", "kurtosis"],
            )
            #add incoming rows data to dataframe
            framey = framey.add(
                {
                    "column name": col,
                    "skewness": scipy.stats.skew(dataframe[col]),
                    "kurtosis": scipy.stats.kurtosis(dataframe[col])
                }
            )
        display(framey)
        display("--------------------")
        ##correlation of the columns
        display("dataframe\'s correlation is :")
        display(dataframe.corr())
        display("--------------------")
        ##covariance of the columns
        display("dataframe\'s covariance is :")
        display(dataframe.cov())
        display("--------------------")
        ## chi scores
        display("dataframe\'s chi scores are :")
        display(dataframe.chisquare())
        display("--------------------")

    except Exception as e:
        print("An error occured: ", e)
        return []

In [23]:
get_stats(adv_df)

'--------------------'

"dataframe's shape is :"

(182, 37)

'--------------------'

"dataframe's columns are :"

...

Index                            132
window_id                       1456
window_start                    1456
packet_count                    1456
bytes_per_second                1456
avg_packet_size                 1456
packet_size_std                 1456
unique_protocols                1456
unique_src_ips                  1456
unique_dst_ips                  1456
unique_src_ports                1456
unique_dst_ports                1456
unique_ip_pairs                 1456
unique_port_pairs               1456
unique_connections              1456
dns_query_count                 1456
bytes_in                        1456
bytes_out                       1456
packets_in                      1456
packets_out                     1456
io_bytes_ratio                  1456
io_packet_ratio                 1456
new_connections                 1456
syn_count                       1456
fin_count                       1456
rst_count                       1456
syn_fin_ratio                   1456
c

'--------------------'

"dataframe's column stats are :"

An error occured:  numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


[]